## **Procesar el Json de `fichas_tecnicas_mapped.json` para poder trabajar con Chatbots**

---
### **Librerías**

In [ ]:
import json
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt

# from sklearn.decomposition import uPCA
import seaborn as sns
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM

/Users/angel/miniconda3/envs/no_str/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-02 23:21:11.908897: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


---
### **1. Procesar JSON para modelo de embeddings**

---
#### **1.1 Load Data**

In [2]:
# Ruta del archivo JSON (ajusta la ruta si es necesario)
json_path = "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json"

# Cargar el JSON
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Mostrar los primeros elementos para revisar la estructura
print(
    json.dumps(data[:2], indent=4, ensure_ascii=False)
)  # Mostramos solo 2 para no saturar

[
    {
        "indicaciones": "en base a su efecto antiagregante plaquetario está indicado en la profilaxis de: infarto de miocardio o reinfarto de miocardio en pacientes con angina de pecho inestable y para prevenir la recurrencia del mismo en pacientes con historia de infarto de miocardio. prevención de la oclusión del bypass aortocoronario. tromboflebitis, flebotrombosis y riesgo de trombosis arteriales. tromboembolismo post-operatorio en pacientes con prótesis vasculares biológicas o con shunts arteriovenosos. tratamiento de ataques isquémicos transitorios en varones con isquemia cerebral transitoria para reducir el riesgo de accidente cerebrovascular. estas indicaciones requieren siempre un control médico.",
        "posologia": "posología: como inhibidor de la agregación plaquetaria: en la prevención de accidentes tromboembólicos (infarto, angina de pecho, prótesis valvulares biológicas, by pass arteriales, tromboflebitis, flebotrombosis y riesgo de trombosis arterial). dosis m

In [3]:
# Mostrar todas las claves de un solo medicamento para entender la estructura
print("Claves disponibles en el JSON:")
print(list(data[0].keys()))

Claves disponibles en el JSON:
['indicaciones', 'posologia', 'contraindicaciones', 'advertencias', 'interacciones', 'fertilidad_embarazo', 'efectos_conducir', 'reacciones_adversas', 'sobredosis', 'ATC', 'Propiedades_farmacocineticas', 'excipientes', 'incompatibilidades', 'precauciones_conservacion', 'fecha_revision', 'nombre_archivo', 'ATC_Nivel_Anatomico', 'Descripcion_Nivel_Anatomico', 'ATC_Nivel_2_Subgrupo_Terapeutico', 'Descripcion_Nivel_2_Subgrupo_Terapeutico', 'ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico', 'Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico', 'ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico', 'Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico', 'ATC_Nivel_5_Principio_Activo', 'Descripcion_Nivel_5_Principio_Activo']


---
#### **1.2 Convertir el JSON en fragmentos**

Queremos convertir el JSON inicial a una estructura tal que así:

```python
[
    {
        "medicamento": "Paracetamol",
        "categoria": "indicaciones",
        "texto": "En base a su efecto antiagregante plaquetario está indicado en..."
    },
    {
        "medicamento": "Ibuprofeno",
        "categoria": "advertencias",
        "texto": "Dado el efecto antiagregante plaquetario del ibuprofeno..."
    },
    ...
]
```

In [ ]:
# Generar lista de fragmentos para el chatbot
# Esta celda recorre cada medicamento del JSON y extrae los campos de interés.
# Si el valor de un campo es None, se reemplaza por una cadena vacía.
# Se elimina la extensión ".txt" del campo "nombre_archivo" para obtener el nombre del medicamento.

# Lista donde almacenaremos los fragmentos
fragments = []

# Definimos los campos de interés que queremos extraer
campos_interes = [
    "indicaciones",
    "posologia",
    "contraindicaciones",
    "advertencias",
    "interacciones",
    "fertilidad_embarazo",
    "efectos_conducir",
    "reacciones_adversas",
    "sobredosis",
    "Propiedades_farmacocineticas",
    "excipientes",
    "incompatibilidades",
    "precauciones_conservacion",
    "Descripcion_Nivel_Anatomico",
]

# Recorremos cada medicamento en el JSON
for medicamento in data:
    # Extraer el nombre del medicamento desde "nombre_archivo" y quitar la extensión ".txt"
    nombre_medicamento = medicamento.get("nombre_archivo", "Desconocido").replace(
        ".txt", ""
    )

    # Iteramos sobre cada campo de interés
    for campo in campos_interes:
        # Obtener el valor del campo; si es None, lo reemplazamos por una cadena vacía
        valor = medicamento.get(campo)
        if valor is None:
            texto = ""
        else:
            texto = valor.strip()  # Eliminamos espacios en blanco innecesarios

        # Solo añadimos el fragmento si hay contenido en el campo (no vacío)
        if texto:
            fragments.append(
                {
                    "medicamento": nombre_medicamento,  # Nombre del medicamento sin ".txt"
                    "categoria": campo,  # Categoría (por ejemplo, "indicaciones", "advertencias", etc.)
                    "texto": texto,  # Contenido del campo
                }
            )

# Mostrar algunos ejemplos de fragmentos generados para verificar la estructura
print("Ejemplo de fragmentos generados:")
print(json.dumps(fragments[:5], indent=4, ensure_ascii=False))

---
### **2. Generación de embeddings con el modelo `Sentence Transformers`**

En esta fase, utilizamos el modelo preentrenado all-MiniLM-L6-v2 para transformar cada fragmento de nuestro JSON en un vector numérico (embedding). Estos embeddings capturan la semántica del texto, lo que nos permite realizar búsquedas basadas en similitud y establecer relaciones entre fragmentos.

Nuestro JSON enriquecido con embeddings funciona como una base de datos vectorial. Esto es esencial en un pipeline de Retrieval-Augmented Generation (RAG), donde se recupera la información relevante y se utiliza como contexto para generar respuestas.

Modelo Generativo Recomendado para el Chatbot
Para la parte generativa del chatbot, recomendamos utilizar un modelo diseñado para integrarse en arquitecturas RAG. Algunas opciones compatibles son:

Facebook RAG-Token (facebook/rag-token-nq):
Diseñado específicamente para integrar recuperación y generación, aprovechando embeddings semánticos.

Flan-T5 o Llama2:
Pueden ser fine-tuned para tareas de RAG y ofrecen excelentes resultados en generación de respuestas.

In [ ]:
# Configuramos PyTorch para usar un número mayor de hilos (ajusta según tu CPU)
torch.set_num_threads(8)  # Por ejemplo, usar 8 hilos (8 cores)

# Cargamos el modelo preentrenado 'all-MiniLM-L6-v2'
model = SentenceTransformer("all-MiniLM-L6-v2")

# Extraemos el texto de cada fragmento
texts = [frag["texto"] for frag in fragments]

# Generamos los embeddings para cada fragmento, aumentando el batch_size para optimizar el procesamiento
embeddings = model.encode(
    texts, convert_to_numpy=True, show_progress_bar=True, batch_size=64
)

# Mostramos la dimensión de los embeddings para confirmar que se han generado correctamente
print("Número de fragmentos:", len(texts))
print("Dimensión de cada embedding:", embeddings.shape[1])

In [ ]:
# Guardamos el array de embeddings en un archivo .npy
output_path = "../../data/outputs/5_chatbot/embeddings.npy"
np.save(output_path, embeddings)

##### **PCA DE CHARLY QUE NO SIRVE PARA NADA**

In [ ]:
# 1. Cargar y comprobar la forma del array de embeddings
embeddings = np.load(output_path)
print("Forma del array de embeddings:", embeddings.shape)

# 2. Mostrar las primeras 5 filas del array
print("Primeras 5 filas del array de embeddings:")
print(embeddings[:5])

# 3. Explicación sobre lo que representa cada vector de embedding
print("\nAnálisis del espacio de embeddings:")
print(
    "Cada vector de embedding es una representación numérica del texto obtenido a partir del modelo de SentenceTransformer."
)
print(
    "Cada dimensión en este vector es una característica latente aprendida durante el entrenamiento del modelo,"
)
print(
    "capaz de captar aspectos semánticos y contextuales del lenguaje. Estas dimensiones no tienen una interpretación"
)
print(
    "directa como 'la dimensión 1 representa X', sino que en conjunto forman un espacio latente que captura la semántica."
)

# 4. Visualización del espacio de embeddings mediante PCA
# Reducimos a 2 dimensiones para graficar
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Imprimir la varianza explicada por cada componente principal
print("\nVarianza explicada por cada componente principal:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"Componente {i+1}: {var*100:.2f}%")

plt.figure(figsize=(10, 7))
sns.scatterplot(x=embeddings_2d[:, 0], y=embeddings_2d[:, 1], s=50, color="dodgerblue")
plt.title("Visualización de embeddings usando PCA")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.tight_layout()
plt.show()

# Comentarios adicionales:
# - La forma del array indica el número total de documentos y la dimensión de cada embedding.
# - Cada posición en un embedding es una característica latente sin significado directo individual, pero su combinación
#   permite una representación semántica del texto.
# - La reducción a dos dimensiones mediante PCA permite identificar las direcciones que capturan mayor varianza,
#   lo cual ayuda a interpretar qué tan dispersos o agrupados están los textos en el espacio latente.
# - Técnicas adicionales como t-SNE o UMAP pueden complementarse para analizar agrupaciones semánticas en los datos.

---
### **3. Almacenar Embeddings en Base de Datos Vectorial (`FAISS`)**

Creamos un índice FAISS usando IndexFlatL2
- Un índice en FAISS es una estructura optimizada para almacenar y buscar embeddings.
- IndexFlatL2 es un tipo de índice simple que usa distancia Euclidiana (L2) para comparar vectores.
- La distancia L2 (Euclidiana) entre dos vectores es la raíz cuadrada de la suma de las diferencias al cuadrado entre sus componentes.
- Se usa para medir la similitud: menor distancia = mayor similitud.



In [ ]:
# 1. Cargamos los embeddings desde el archivo .npy
embeddings_path = "../../data/outputs/5_chatbot/embeddings.npy"
embeddings = np.load(embeddings_path)

# Creamos el índice FAISS con la dimensión de los embeddings
embedding_dim = embeddings.shape[1]  # Número de dimensiones de cada embedding
index = faiss.IndexFlatL2(embedding_dim)  # Creamos el índice basado en L2

# 3. Insertamos los embeddings en el índice
index.add(embeddings)  # Agregamos los vectores al índice

# 4. Guardamos el índice en un archivo binario para futuras consultas
faiss.write_index(index, "../../data/outputs/5_chatbot/faiss_index.bin")

print("Índice FAISS creado y guardado correctamente.")

In [ ]:
# !pip install faiss-cpu
# Cpu es más que suficiente para este caso, pero si se quiere usar GPU se puede instalar faiss-gpu

---
### **4. Búsqueda en FAISS -- Hasta aquí más o menos revisado, resto no mirar**

Ahora que tenemos el índice FAISS creado, podemos usarlo para buscar los fragmentos más relevantes para una consulta.

**Pasos de la búsqueda**
1. **Cargamos el índice FAISS** desde el archivo guardado.
2. **Cargamos los fragmentos de texto originales** para poder recuperar la información relevante.
3. **Convertimos la consulta en un embedding** usando el mismo modelo `all-MiniLM-L6-v2`.
4. **Buscamos los embeddings más cercanos en FAISS** usando la distancia L2.
5. **Recuperamos los fragmentos de texto asociados** a los embeddings más cercanos.
6. **Devolvemos los fragmentos más relevantes** como resultados.

*Ejemplo de búsqueda**
Si la consulta del usuario es:
> "¿Cuáles son las contraindicaciones del paracetamol?"

FAISS devolverá los fragmentos más cercanos, como:
> 

Estos fragmentos se pueden usar como contexto para generar respuestas en **Llama 2** dentro de un sistema **RAG (Retrieval-Augmented Generation)**.

In [ ]:
# Cargar el índice FAISS
index_path = "../../data/outputs/5_chatbot/faiss_index.bin"
index = faiss.read_index(index_path)

# Cargar los datos originales (fragmentos de texto)
json_path = "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json"
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Lista donde almacenaremos los fragmentos
fragments = []

# Definimos los campos de interés que queremos extraer
campos_interes = [
    "indicaciones",
    "posologia",
    "contraindicaciones",
    "advertencias",
    "interacciones",
    "fertilidad_embarazo",
    "efectos_conducir",
    "reacciones_adversas",
    "sobredosis",
    "Propiedades_farmacocineticas",
    "excipientes",
    "incompatibilidades",
    "precauciones_conservacion",
]

# Recorremos cada medicamento en el JSON
for medicamento in data:
    # Extraer el nombre del medicamento desde "nombre_archivo" y quitar la extensión ".txt"
    nombre_medicamento = medicamento.get("nombre_archivo", "Desconocido").replace(
        ".txt", ""
    )

    # Iteramos sobre cada campo de interés
    for campo in campos_interes:
        # Obtener el valor del campo; si es None, lo reemplazamos por una cadena vacía
        valor = medicamento.get(campo)
        if valor is None:
            texto = ""
        else:
            texto = valor.strip()  # Eliminamos espacios en blanco innecesarios

        # Solo añadimos el fragmento si hay contenido en el campo (no vacío)
        if texto:
            fragments.append(
                {
                    "medicamento": nombre_medicamento,  # Nombre del medicamento sin ".txt"
                    "categoria": campo,  # Categoría (por ejemplo, "indicaciones", "advertencias", etc.)
                    "texto": texto,  # Contenido del campo
                }
            )

# Cargar el modelo de embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")


# Función para buscar en FAISS
def search_faiss(query, k=5):
    """
    Realiza una búsqueda en FAISS para encontrar los fragmentos más similares a la consulta.

    Parámetros:
    - query (str): La consulta en lenguaje natural.
    - k (int): Número de resultados a recuperar.

    Retorna:
    - Lista de fragmentos de texto relevantes.
    """
    # Convertir la consulta en embedding
    query_embedding = model.encode(query, convert_to_numpy=True).reshape(1, -1)

    # Buscar los k embeddings más cercanos
    distances, indices = index.search(query_embedding, k)

    # Recuperar los fragmentos correspondientes, incluyendo las distancias
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(fragments):  # Asegurar que el índice es válido
            results.append(
                {
                    **fragments[idx],  # Añadir los datos del fragmento
                    "distance": distances[0][i],  # Añadir la distancia de similitud
                }
            )

    return results


# Prueba de búsqueda
query = "¿Cuáles son las contraindicaciones del paracetamol?"
resultados = search_faiss(query, k=10)

# Mostrar resultados
for i, res in enumerate(resultados):
    print(f"Resultado {i+1}:")
    print(f"Medicamento: {res['medicamento']}")
    print(f"Categoría: {res['categoria']}")
    print(f"Texto: {res['texto']}\n")

Resultado 1:
Medicamento: PARACETAMOL_MABO_1_g_COMPRIMIDOS_EFG
Categoría: contraindicaciones
Texto: hipersensibilidad al paracetamol, o a alguno de los excipientes.

Resultado 2:
Medicamento: HIDONAC_ANTIDOTO_200_MG_ML_CONCENTRADO_PARA_SOLUCION_PARA_PERFUSION
Categoría: contraindicaciones
Texto: no existen contraindicaciones en el tratamiento de la sobredosis de paracetamol.

Resultado 3:
Medicamento: ANTIDOL_NOCHE_500_MG_25_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA
Categoría: contraindicaciones
Texto: - - hipersensibilidad al paracetamol, a difenhidramina o a alguno de los excipientes incluidos en la sección 6.1. porfiria.

Resultado 4:
Medicamento: DOLOMIDINA_500_25MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA
Categoría: contraindicaciones
Texto: - - hipersensibilidad al paracetamol, a difenhidramina o a alguno de los excipientes incluidos en la sección 6.1. porfiria.

Resultado 5:
Medicamento: DOLOSTOP_1_G_COMPRIMIDOS
Categoría: contraindicaciones
Texto: hipersensibilidad al paracetamol o a

### **5. Cargar el modelo generativo (Llama 2)**

In [ ]:
"""def load_llama_model():
# Define el nombre del modelo; asegúrate de tener acceso a los pesos
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# Podemos cargar otros modelos alternativos que no requieran autenticación
model_name = "google/flan-t5-large"  # Alternativa: "facebook/opt-1.3b" o "EleutherAI/gpt-neo-1.3B"


# Carga el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Para reducir el uso de memoria
    device_map="auto"           # Distribuir modelo entre dispositivos disponibles
)

return tokenizer, model'
"""

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


def load_language_model():
    # Modelo Flan-T5 (modelo secuencia a secuencia)
    model_name = "google/flan-t5-large"

    # Carga el tokenizer y el modelo con la clase correcta
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(  # Para modelos T5
        model_name, torch_dtype=torch.float16, device_map="auto"
    )

    return tokenizer, model

### **6.Función para generar respuestas basadas en los fragmentos recuperados**

In [ ]:
def generate_answer(query, retrieved_fragments, tokenizer, model):
    """
    Genera una respuesta basada en los fragmentos recuperados usando Llama 2.

    Parámetros:
    - query (str): La consulta del usuario
    - retrieved_fragments (list): Lista de fragmentos recuperados
    - tokenizer: Tokenizador del modelo
    - model: Modelo generativo

    Retorna:
    - str: Respuesta generada
    """
    # Formatea los fragmentos recuperados como contexto
    context = ""
    for i, frag in enumerate(
        retrieved_fragments[:3]
    ):  # Limitamos a 3 fragmentos para no exceder el contexto
        context += f"\nFragmento {i+1}:\n"
        context += f"Medicamento: {frag['medicamento']}\n"
        context += f"Categoría: {frag['categoria']}\n"
        context += (
            f"Información: {frag['texto'][:500]}...\n"
            if len(frag["texto"]) > 500
            else f"Información: {frag['texto']}\n"
        )

    # Construye el prompt para el modelo
    prompt = f"""Eres un asistente médico especializado en información sobre medicamentos.
Basándote únicamente en la siguiente información sobre medicamentos:

{context}

Responde de manera clara y precisa a esta pregunta: {query}

Si la información proporcionada no es suficiente para responder completamente, indica qué datos faltan.
Tu respuesta debe ser:
1. Precisa y basada solo en el contexto proporcionado
2. Estructurada y fácil de entender
3. Sin añadir información que no esté en los fragmentos
4. Con referencias claras al medicamento mencionado

Respuesta:"""

    # Tokeniza el prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Si hay GPU disponible, mueve los tensores a la GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_ids = input_ids.to(device)

    # Genera la respuesta
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + 500,  # Limita la longitud de salida
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
        )

    # Decodifica la respuesta generada
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extrae solo la parte de la respuesta después del prompt
    response = response[
        len(prompt) - 10 :
    ]  # Ajuste para capturar la respuesta correctamente

    return response.strip()

### **7. Sistema RAG completo**

In [ ]:
def medicamentos_rag(query):
    """
    Sistema RAG completo para consultas sobre medicamentos.

    Parámetros:
    - query (str): Consulta del usuario

    Retorna:
    - str: Respuesta generada
    - list: Fragmentos recuperados
    """
    # 1. Recuperar fragmentos relevantes
    print("Buscando información relevante...")
    retrieved_fragments = search_faiss(query, k=5)

    # 2. Cargar modelo generativo (en un sistema de producción, esto se haría una sola vez)
    print("Preparando el modelo generativo...")
    tokenizer, model = load_language_model()

    # 3. Generar respuesta basada en los fragmentos
    print("Generando respuesta...")
    response = generate_answer(query, retrieved_fragments, tokenizer, model)

    return response, retrieved_fragments

### **8. Demostración del sistema**

In [ ]:
def demo_rag():
    query = input("Introduce tu consulta sobre medicamentos: ")

    response, fragments = medicamentos_rag(query)

    print("\n=== Fragmentos Recuperados ===")
    for i, frag in enumerate(fragments[:3]):
        print(
            f"Fragmento {i+1}: {frag['medicamento']} - {frag['categoria']} (Similitud: {1 - frag['distance']:.4f})"
        )

    print("\n=== Respuesta Generada ===")
    print(responsegg)

    return response, fragments

In [15]:
#### EJEMPLO DE USO:
response, retrieved_docs = demo_rag()

Buscando información relevante...
Preparando el modelo generativo...
Generando respuesta...

=== Fragmentos Recuperados ===
Fragmento 1: TIGETEMEL_500_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA_EFG - contraindicaciones (Similitud: 0.4884)
Fragmento 2: PARACETAMOL_MABO_1_g_COMPRIMIDOS_EFG - contraindicaciones (Similitud: 0.4622)
Fragmento 3: PARACETAMOL_CAFEINA_CHANELLE_500_MG_65_MG_COMPRIMIDOS_RECUBIERTOS_CON_PELICULA_EFG - contraindicaciones (Similitud: 0.4503)

=== Respuesta Generada ===



In [16]:
response

''